# Retrieval comments from topics

In [170]:
import urllib
from bs4 import BeautifulSoup
import time
import re
import pandas as pd

In [2]:
quattroruote_links = ['https://forum.quattroruote.it/threads/porsche-911-992.111443/',
                     'https://forum.quattroruote.it/threads/porsche-718-cayman.103817/',
                     'https://forum.quattroruote.it/threads/ecco-la-boxster-718-a-4-cilindri.102011/',
                     'https://forum.quattroruote.it/threads/non-acquistate-porsche.75732/',
                     'https://forum.quattroruote.it/threads/nessuno-parla-della-997-gt3-rs-4-0.50248/',
                     'https://forum.quattroruote.it/threads/cayman-2-7-245cv.52952/',
                     'https://forum.quattroruote.it/threads/porsche-cayenne.8981/',
                     'https://forum.quattroruote.it/threads/porsche-addio-alle-versioni-diesel.118611/',
                     'https://forum.quattroruote.it/threads/nuovo-possessore-porsche-cayman.109681/',
                     'https://forum.quattroruote.it/threads/991.54988/',
                     ]

In [3]:
bmwpassion_links = ['https://www.bmwpassion.com/forum/threads/porsche-panamera.39732/',
                   'https://www.bmwpassion.com/forum/threads/difettosit%C3%A0-porsche.135186/',
                   'https://www.bmwpassion.com/forum/threads/porsche-panamera.13257/',
                   'https://www.bmwpassion.com/forum/threads/porsche-boxster.9182/',
                   'https://www.bmwpassion.com/forum/threads/porsche-911-turbo-o-porsche-911-gt3.27795/',
                   'https://www.bmwpassion.com/forum/posts/177476/',
                   'https://www.bmwpassion.com/forum/threads/porsche-superata.45802/',
                   'https://www.bmwpassion.com/forum/threads/porsche-addio-diesel.326375/',
                   ]

In [4]:
porschemania_links = ['http://www.porschemania.it/discus/messages/10548/849560.html?1559115371',
                     'http://www.porschemania.it/discus/messages/10548/848801.html?1559049684',
                     'http://www.porschemania.it/discus/messages/10548/830159.html?1548146413',
                     'http://www.porschemania.it/discus/messages/502416/853834.html?1554300972',
                     'http://www.porschemania.it/discus/messages/453055/483409.html?1563501636',
                     'http://www.porschemania.it/discus/messages/453055/805145.html?1510769697',
                     'http://www.porschemania.it/discus/messages/453055/783843.html?1486350841',
                     'http://www.porschemania.it/discus/messages/453055/567652.html?1309903765',
                     'http://www.porschemania.it/discus/messages/453055/839297.html?1541411368',
                     'http://www.porschemania.it/discus/messages/96836/846938.html?1543829504',]

In [89]:
def clean_date(date):
    months_1 = {
        'gennaio': '01',
        'febbraio': '02',
        'marzo': '03',
        'aprile': '04',
        'maggio': '05',
        'giugno': '06',
        'luglio': '07',
        'agosto': '08',
        'settembre': '09',
        'ottobre': '10',
        'novembre': '11',
        'dicembre': '12'
    }
    months_2 = {
        'gen': '01',
        'feb': '02',
        'mar': '03',
        'apr': '04',
        'mag': '05',
        'giu': '06',
        'lug': '07',
        'ago': '08',
        'set': '09',
        'ott': '10',
        'nov': '11',
        'dic': '12'
    }
    if len(re.findall(r'^[0-9]{4}-[0-9]{2}-[0-9]{2}T[0-9]{2}:[0-9]{2}:[0-9]{2}\+[0-9]{4}$', date)) > 0:
        # autopareri
        date = re.sub(r'T', ' ', date)
        date = re.sub(r'\+.*', '', date)
    elif len(re.findall(r'^[0-9]+\s[a-zA-Z]+\s[0-9]{4}$', date)) > 0:
        # quattroruote + bmwpassion
        tokens = date.split()
        Y = tokens[2]
        D = tokens[0]
        M = months_1[tokens[1].lower()]
        if len(D) == 1:
            D = '0' + D
        date = Y + '-' + M + '-' + D + ' ' + '00:00:00'
    elif len(re.findall(r'^[0-9]{2}\s[a-zA-Z]+\s[0-9]{4}\s@\s[0-9].:[0-9].\s*$', date)) > 0:
        # hdmotori
        tokens = date.split()
        D = tokens[0]
        Y = tokens[2]
        M = months_2[tokens[1].lower()]
        h = tokens[4]
        date = Y + '-' + M + '-' + D + ' ' + h + ':00'
    elif len(re.findall(r'^[a-zA-Zì]*\s[0-9]{2}\s[a-zA-Z]*\s[0-9]{4}\s-\s[0-9].:[0-9].\s*$', date)) > 0:
        # porschemania
        tokens = date.split()
        D = tokens[1]
        M = months_1[tokens[2].lower()]
        Y = tokens[3]
        h = tokens[5]
        date = Y + '-' + M + '-' + D + ' ' + h + ':00'
    elif len(re.findall(r'^[0-9]{2}/[0-9]{2}/[0-9]{4},\s[0-9].:[0-9].\s*$', date)) > 0:
        # forumelettrico
        tokens = date.split()
        tok = tokens[0].split('/')
        D = tok[0]
        M = tok[1]
        Y = tok[2][:-1]
        h = tokens[1]
        date = Y + '-' + M + '-' + D + ' ' + h + ':00'
    return date

def clean_tweet(tweet):
    # add one space after punctation
    #tweet = re.sub('([.,!?()])', r'', tweet)
    # remove character escapes
    tweet = re.sub(r'(\\n|\\t|\\r)', ' ', tweet)
    # remove 2+ spaces
    tweet = re.sub(r'\s{2,}', ' ', tweet)
    # remove initial space
    tweet = re.sub(r'^\s+', '', tweet)
    return tweet

def clean_quote(quote):
    quote = str(quote)
    quote = re.sub(r'[\.\,\(\)]', '', quote)
    quote = re.sub(r'^\[', '', quote)
    quote = re.sub(r'\]$', '', quote)
    quote = re.sub(r'(\\n|\\t|\\r)', ' ', quote)
    quote = re.sub(r'\s{2,}', ' ', quote)
    quote = re.sub(r'^.{,20}ha scritto: ', '', quote)
    quote = re.sub(r'Clicca per allargare', '', quote)
    return quote

Informazioni da prendere:
<ul>
    <li>Nome Forum</li>
    <li>URL commento</li>
    <li>Titolo Thread</li>
    <li>Timestamp commento</li>
    <li>Nome utente</li>
    <li>Num messaggi utente</li>
    <li>Data iscrizione utente</li>
    <li>da dove scrive</li>
    <li>testo</li>
    <li>quote</li>
</ul>

### Quattroruote

In [57]:
# get info from message
def quattroruote_data_from_message(message):
    # username
    username = message.findAll('a', 'username')[0].getText()
    
    user_info = message.findAll('dl', 'pairsJustified')
    # num messages
    num_messages = int(user_info[1].dd.getText().replace('.', ''))
    # subscription date
    sub_date = time.strftime('%d-%m-%Y %H:%M:%S', time.localtime(int(user_info[0].dd.getText())))
    
    # datetime
    datetime_html = message.findAll('span', 'DateTime')
    if len(datetime_html) == 0:
        datetime_html = message.findAll('abbr', 'DateTime')
    datetime = datetime_html[0].get('title')
    
    # FORMAT DATETIME DD-MM-YYYY HH:MM:SS
    datetime = datetime.lower()
    # 31 Marzo 2012 alle 23:25
    tokens = datetime.split()
    day = tokens[0]
    month = tokens[1].replace('gennaio', '01').replace('febbraio', '02').replace('marzo', '03').replace('aprile', '04').replace('maggio', '05').replace('giugno', '06').replace('luglio', '07').replace('agosto', '08').replace('settembre', '09').replace('ottobre', '10').replace('novembre', '11').replace('dicembre', '12')
    year = tokens[2]
    hour = tokens[4]
    datetime = day + '-' + month + '-' + year + ' ' + hour + ':00'
    
    
    # quotes
    html_quotes = message.findAll('div', 'quote')
    quotes = []
    for q in html_quotes:
        quotes.append(q.getText())
    # text
    for tag in message.findAll('div', ['quote', 'attribution', 'quoteExpand']):
        tag.replaceWith('')
    text = message.findAll('blockquote', 'messageText')[0].getText().replace('\n', ' ').replace('\t', ' ').replace('\s', ' ')
    # images
    html_images = message.findAll('img', 'bbCodeImage')
    for img in html_images:
        text = text + str(img) + '\n'
       
    return username, sub_date, num_messages, datetime, quotes, text

In [91]:
quattroruote_base_link = 'https://forum.quattroruote.it/'

quattroruote_messages = []

i = 0
for link in quattroruote_links:
    i += 1
    queue = [link]
    while len(queue) > 0:
        seed = queue.pop(0)
        print('######## ' + str(i) + ' - ' + seed + ' ########')
        try:
            with urllib.request.urlopen(seed) as response:
                html = response.read()
                soup = BeautifulSoup(html, 'html.parser')
            # get all a tags with text 'Successiva >' and put the link into list
            for a in soup.findAll('a'):
                if a.getText() == 'Successiva >':
                    queue.append(quattroruote_base_link + a.get('href'))
                    break

            # title and url info
            thread_title = soup.findAll('div', 'titleBar')[0].h1.getText()
            thread_url = seed
            # get all plain html messages and save in a list
            messages = soup.findAll('li', 'message')
            print(str(len(messages)) + ' messages')
            for message in messages:
                username, sub_date, num_messages, datetime, quotes, text = quattroruote_data_from_message(message)
                quattroruote_messages.append(('Quattroruote', str(thread_title), str(thread_url), str(datetime), str(username), str(sub_date), str(num_messages), clean_quote(str(quotes)), clean_tweet(str(text))))
        except:
            print('error')
            continue

######## 1 - https://forum.quattroruote.it/threads/porsche-911-992.111443/ ########
15 messages
######## 1 - https://forum.quattroruote.it/threads/porsche-911-992.111443/page-2 ########
15 messages
######## 1 - https://forum.quattroruote.it/threads/porsche-911-992.111443/page-3 ########
15 messages
######## 1 - https://forum.quattroruote.it/threads/porsche-911-992.111443/page-4 ########
13 messages
error
######## 2 - https://forum.quattroruote.it/threads/porsche-718-cayman.103817/ ########
15 messages
######## 2 - https://forum.quattroruote.it/threads/porsche-718-cayman.103817/page-2 ########
15 messages
######## 2 - https://forum.quattroruote.it/threads/porsche-718-cayman.103817/page-3 ########
15 messages
######## 2 - https://forum.quattroruote.it/threads/porsche-718-cayman.103817/page-4 ########
6 messages
######## 3 - https://forum.quattroruote.it/threads/ecco-la-boxster-718-a-4-cilindri.102011/ ########
15 messages
######## 3 - https://forum.quattroruote.it/threads/ecco-la-boxster

In [92]:
len(quattroruote_messages)

426

### Bmwpassion

In [148]:
# get info from message
def bmwpassion_data_from_message(message):
    # username
    username = message.findAll('a', 'username')[0].getText()
    # datetime 14 Aprile 2007 alle 22:33
    datetime = message.findAll('span', 'DateTime')[0].get('title').lower().replace('gennaio', '01').replace('febbraio', '02').replace('marzo', '03').replace('aprile', '04').replace('maggio', '05').replace('giugno', '06').replace('luglio', '07').replace('agosto', '08').replace('settembre', '09').replace('ottobre', '10').replace('novembre', '11').replace('dicembre', '12')
    tokens = datetime.split()
    datetime = tokens[0] + '-' + tokens[1] + '-' + tokens[2] + ' ' + tokens[4] + ':00'
        
    user_info = message.findAll('dl', 'pairsJustified')
    
    #num messages
    num_messages = int(user_info[0].dd.a.getText().replace('.', ''))

    # subscription date
    sub_date = user_info[2].dd.getText().lower().replace('gennaio', '01').replace('febbraio', '02').replace('marzo', '03').replace('aprile', '04').replace('maggio', '05').replace('giugno', '06').replace('luglio', '07').replace('agosto', '08').replace('settembre', '09').replace('ottobre', '10').replace('novembre', '11').replace('dicembre', '12')
    tokens = sub_date.split()
    sub_date = tokens[0] + '-' + tokens[1] + '-' + tokens[2]
    
    # quotes
    html_quotes = message.findAll('div', 'quote')
    quotes = []
    for q in html_quotes:
        quotes.append(q.getText())
    # text
    for tag in message.findAll('aside'):
        tag.replaceWith('')
    #text = message.getText().replace('\n', ' ').replace('\t', ' ').replace('\s', ' ')
    text = soup.findAll('blockquote', 'messageText')[0].text.replace('\n', '').replace('\t', '').replace('\s', '')
    # images
    html_images = []#m.findAll('img', 'bbCodeImage')
    #for img in html_images:
    #    text = text + str(img) + '\n'
       
    return username, sub_date, num_messages, datetime, quotes, text

In [154]:
bmwpassion_base_link = 'https://www.bmwpassion.com/forum/'
bmwpassion_messages = []

i = 0
for link in bmwpassion_links:
    i += 1
    queue = [link]
    while len(queue) > 0:
        seed = queue.pop(0)
        print('######## ' + str(i) + ' - ' + seed + ' ########')
        try:
            with urllib.request.urlopen(seed) as response:
                html = response.read()
                soup = BeautifulSoup(html, 'html.parser')
            # get all a tags with text 'Succ. >' and put the link into list
            for a in set(soup.findAll('a', 'text')):
                if a.getText() == 'Succ. >':
                    queue.append(bmwpassion_base_link + a.get('href'))
                    break
            # title and url info
            thread_title = soup.find_all('h1')[0].getText().replace('\n', '').replace('\t', '')
            thread_url = seed
            # get all plain html messages and save in a list
            messages = soup.findAll('li', 'message')
            print(str(len(messages)) + ' messages')
            for message in messages:
                username, sub_date, num_messages, datetime, quotes, text = bmwpassion_data_from_message(message)
                bmwpassion_messages.append(('Bmwpassion', str(thread_title), str(thread_url), str(datetime), str(username), str(sub_date), str(num_messages), clean_quote(str(quotes)), clean_tweet(str(text))))
        except:
            print('error')
            continue

######## 1 - https://www.bmwpassion.com/forum/threads/porsche-panamera.39732/ ########
20 messages
######## 1 - https://www.bmwpassion.com/forum/threads/porsche-panamera.39732/page-2 ########
3 messages
######## 2 - https://www.bmwpassion.com/forum/threads/difettosit%C3%A0-porsche.135186/ ########
20 messages
######## 2 - https://www.bmwpassion.com/forum/threads/difettosit%C3%A0-porsche.135186/page-2 ########
4 messages
######## 3 - https://www.bmwpassion.com/forum/threads/porsche-panamera.13257/ ########
3 messages
######## 4 - https://www.bmwpassion.com/forum/threads/porsche-boxster.9182/ ########
14 messages
######## 5 - https://www.bmwpassion.com/forum/threads/porsche-911-turbo-o-porsche-911-gt3.27795/ ########
20 messages
######## 5 - https://www.bmwpassion.com/forum/threads/porsche-911-turbo-o-porsche-911-gt3.27795/page-2 ########
20 messages
######## 5 - https://www.bmwpassion.com/forum/threads/porsche-911-turbo-o-porsche-911-gt3.27795/page-3 ########
18 messages
######## 6 - ht

In [153]:
print(len(bmwpassion_messages))

883


### Porschemania

In [167]:
porschemania_messages = []

j = 0
for seed in porschemania_links:
    j += 1
    print('######## ' + str(j) + ' - ' + seed + ' ########')
    with urllib.request.urlopen(seed) as response:
        html = response.read()
        soup = BeautifulSoup(html, 'html.parser')
    #title
    thread_title = soup.title.getText().replace('PorscheMania Forum: ', '')
    #username
    usernames_list = re.findall(r"<!--field:uname-->(\w*)<!--/field-->", str(html))
    
    # subscription 05-2005
    sub_date_list = re.findall(r"<!--field:registered-->([\w-]*)<!--/field-->", str(html))
    for i in range(len(sub_date_list)):
        sub_date_list[i] = '01-' + (sub_date_list[i].replace('"', ''))
    
    # num messages
    num_messages_list = re.findall(r"<!--field:posts-->(\w*)<!--/field-->", str(html))
    for i in range(len(num_messages_list)):
        num_messages_list[i] = num_messages_list[i].replace('"', '')    
    
    #date
    datetime_list = []
    for dt in re.findall(r"Inviato il (.*?): &nbsp;&nbsp;", str(html), re.MULTILINE):
        datetime_list.append(dt.replace('\\xec', 'ì'))
    #text
    comments_html_list = re.findall(r"<!--Text-->(.*?)<!--/Text-->", str(html), re.MULTILINE)
    quotes_list = []
    comments_list = []
    for comment in comments_html_list:
        c = BeautifulSoup(comment, 'html.parser')
        comm_html = c.findAll('blockquote')
        if len(comm_html) > 0:
            quotes_list.append(comm_html[0].i.getText())
        else:
            quotes_list.append('')
        for tag in c.findAll('blockquote'):
            tag.replaceWith('')
        comments_list.append(c.getText())
        
    print(str(len(porschemania_messages)) + ' messages') 
    
    for i in range(len(usernames_list)):
        porschemania_messages.append(('Porschemania', str(thread_title), str(seed), str(datetime_list[i]), str(usernames_list[i]), str(sub_date_list[i]), str(num_messages_list[i]), str(quotes_list[i]), str(comments_list[i])))

######## 1 - http://www.porschemania.it/discus/messages/10548/849560.html?1559115371 ########
0 messages
######## 2 - http://www.porschemania.it/discus/messages/10548/848801.html?1559049684 ########
174 messages
######## 3 - http://www.porschemania.it/discus/messages/10548/830159.html?1548146413 ########
385 messages
######## 4 - http://www.porschemania.it/discus/messages/502416/853834.html?1554300972 ########
559 messages
######## 5 - http://www.porschemania.it/discus/messages/453055/483409.html?1563501636 ########
610 messages
######## 6 - http://www.porschemania.it/discus/messages/453055/805145.html?1510769697 ########
806 messages
######## 7 - http://www.porschemania.it/discus/messages/453055/783843.html?1486350841 ########
851 messages
######## 8 - http://www.porschemania.it/discus/messages/453055/567652.html?1309903765 ########
915 messages
######## 9 - http://www.porschemania.it/discus/messages/453055/839297.html?1541411368 ########
959 messages
######## 10 - http://www.porschem

In [168]:
print(len(porschemania_messages))

1048


### Write CSV

In [171]:
labels = ['Forum', 'Thread_Title', 'URL', 'Timestamp', 'Username', 'Subscription_Date', 'Num_Messages', 'Quote', 'Text']
df = pd.DataFrame.from_records(quattroruote_messages + bmwpassion_messages + porschemania_messages, columns=labels)

In [174]:
df

,Forum,Thread_Title,URL,Timestamp,Username,Subscription_Date,Num_Messages,Quote,Text
0,Quattroruote,Porsche 911/992,https://forum.quattroruote.it/threads/porsche-...,19-07-2017 01:26:00,pilota54,03-04-2009 02:01:04,27259,,Stanno circolando sul web le prime immagini di...
1,Quattroruote,Porsche 911/992,https://forum.quattroruote.it/threads/porsche-...,10-04-2018 18:06:00,pilota54,03-04-2009 02:01:04,27259,,Dovrebbe essere presentata entro l'anno la Por...
2,Quattroruote,Porsche 911/992,https://forum.quattroruote.it/threads/porsche-...,11-04-2018 20:16:00,Mastertanto,01-04-2009 03:16:07,3582,'Dovrebbe essere presentata entro l\'anno la P...,"A me va benissimo. E mantiene il vecchio, caro..."
3,Quattroruote,Porsche 911/992,https://forum.quattroruote.it/threads/porsche-...,11-04-2018 20:27:00,Mastertanto,01-04-2009 03:16:07,3582,' ma con variazioni più significative rispetto...,Per fortuna. Perché quello sarebbe un po' sven...
4,Quattroruote,Porsche 911/992,https://forum.quattroruote.it/threads/porsche-...,12-04-2018 00:50:00,pilota54,03-04-2009 02:01:04,27259,,"Si, sulla 911 di serie il motore centrale non ..."
5,Quattroruote,Porsche 911/992,https://forum.quattroruote.it/threads/porsche-...,19-04-2018 14:03:00,pilota54,03-04-2009 02:01:04,27259,,"Proseguono i test al ""Ring"" della 992, in part..."
6,Quattroruote,Porsche 911/992,https://forum.quattroruote.it/threads/porsche-...,25-06-2018 18:09:00,pilota54,03-04-2009 02:01:04,27259,,"Ancora test al ""Ring"" per l'attesa Porsche 911..."
7,Quattroruote,Porsche 911/992,https://forum.quattroruote.it/threads/porsche-...,7-07-2018 11:43:00,Mastertanto,01-04-2009 03:16:07,3582,"'Ancora test al ""Ring"" per l\'attesa Porsche 9...",Io sono convinto che in ogni caso difficilment...
8,Quattroruote,Porsche 911/992,https://forum.quattroruote.it/threads/porsche-...,7-07-2018 11:58:00,pilota54,03-04-2009 02:01:04,27259,,Ame della 911 (le versioni aspirate) quello ch...
9,Quattroruote,Porsche 911/992,https://forum.quattroruote.it/threads/porsche-...,7-07-2018 15:30:00,LA MAGA,18-05-2018 23:35:16,1075,,chissà se ce la faranno a mettere il navigator...


In [175]:
df.to_csv('unlabeled_comments.csv')